# Export skin temperature along the transect on 16 and 19 July 2022
- This script is used to export daytime and nighttime skin temperature along the transect on 16 and 19 July 2022.
- Simulations: GM_SLUCM, GM_CLMU. 
- Reference data: MODIS, VIIRS.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import rasterio
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
model_list = ['wrf', 'wrf-ctsm']
time_list = ['day', 'night']

In [3]:
ds_template = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf/wrfout_d01_2021-12-25_00:00:00.nc')
lons = ds_template['XLONG'][0,0,:].values
lats = ds_template['XLAT'][0,:,0].values
lats

array([53.27471 , 53.28549 , 53.296276, 53.30706 , 53.317844, 53.328636,
       53.339424, 53.350204, 53.360992, 53.371773, 53.38256 , 53.39335 ,
       53.40413 , 53.414917, 53.425705, 53.43649 , 53.447273, 53.45806 ,
       53.468845, 53.47963 , 53.49041 , 53.501198, 53.511986, 53.52277 ,
       53.53355 , 53.544334, 53.555122, 53.565907, 53.576694, 53.587475,
       53.598255, 53.60904 , 53.61983 , 53.630615, 53.641396, 53.652176,
       53.662964, 53.67375 , 53.684532, 53.69532 , 53.7061  , 53.716885,
       53.72767 , 53.738453, 53.749233], dtype=float32)

In [4]:
# find the nearest grid point
centroid_lat = 53.47941
centroid_lon = -2.24369 # the city concil of Manchester
lat_idx = np.abs(lats - centroid_lat).argmin()
lon_idx = np.abs(lons - centroid_lon).argmin()
print(lat_idx, lon_idx)

19 30


In [7]:
max_lat = len(lats)
max_lon = len(lons)
southwest_index_list = []
steps = 40
for i in range(-steps, steps + 1):
    southwest_lat_idx = lat_idx + i
    southwest_lon_idx = lon_idx + i
    if 0 <= southwest_lat_idx < max_lat and 0 <= southwest_lon_idx < max_lon:
        southwest_index_list.append((southwest_lat_idx.item(), southwest_lon_idx.item()))
     
southwest_index_list
with open('./data_for_figure/southwest_index_list.txt', 'w') as f:
    for item in southwest_index_list:
        f.write(f"{item[0]}, {item[1]}\n")

In [8]:
df_section= pd.DataFrame()
df_section['lat'] = [lats[i].item() for i, j in southwest_index_list]
df_section['lon'] = [lons[j].item() for i, j in southwest_index_list]
df_section['index_lat'] = [i for i, j in southwest_index_list]
df_section['index_lon'] = [j for i, j in southwest_index_list]
df_section.to_csv('./data_for_figure/section_info.csv', index=False)
df_section.head()

,lat,lon,index_lat,index_lon
0,53.274712,-2.582367,0,11
1,53.285488,-2.564331,1,12
2,53.296276,-2.546295,2,13
3,53.307060,-2.528259,3,14
4,53.317844,-2.510223,4,15


In [24]:
data_list = []
col_num = 4
row_num = 2
product_list = ['modis', 'viirs']
date = '07-16'
date2 = '16_07'
#date = '07-19'
#date2 = '19_07'
for i in range(col_num*row_num):
    idx_time = i // col_num
    time = time_list[idx_time]
    idx_model = i % col_num
    if idx_model < 2:
        model = model_list[idx_model]    
        ds_date = xr.open_dataset(f'../skin_temperature/data_for_figure/{model}_{time}_{date}.nc')
        tsk_section = []
        for i in range(len(df_section['index_lat'])):
            tsk_point = ds_date['TSK'].isel(south_north=df_section['index_lat'][i], west_east=df_section['index_lon'][i]).item()
            tsk_section.append(tsk_point)
        data_list.append(tsk_section)    
    else:
        product_name = product_list[idx_model - 2]
        tsk_section = []
        with rasterio.open(f'../skin_temperature/data_for_figure/{product_name}_lst_{time}_{date2}_2022_MCR.tif') as src:
            coords = list(zip(df_section['lon'], df_section['lat']))
            for val in src.sample(coords):
                if product_name == 'modis':
                    data = float(val[0] * 0.02 - 273.15)
                    data = np.where(data > 0, data, np.nan)  
                else:
                    data = float(val[0] -273.15)
                tsk_section.append(data)        
        data_list.append(tsk_section) 
df_data_list = pd.DataFrame(data_list).T
df_data_list['lat'] = df_section['lat']
df_data_list['lon'] = df_section['lon']
df_data_list['index_lat'] = df_section['index_lat']
df_data_list['index_lon'] = df_section['index_lon']
R = 6371.0            # mean Earth radius [km]

lat_rad  = np.radians(df_data_list['lat'].to_numpy())
lon_rad  = np.radians(df_data_list['lon'].to_numpy())
# Calculate distance from the centroid point
df_data_list_city_center = df_data_list[(df_data_list['index_lat'] == 19) & (df_data_list['index_lon'] == 30)].copy()
lat0_rad = np.radians(df_data_list_city_center['lat'].item())
lon0_rad = np.radians(df_data_list_city_center['lon'].item())

dlat = lat_rad - lat0_rad
dlon = lon_rad - lon0_rad

a = np.sin(dlat/2)**2 + np.cos(lat0_rad)*np.cos(lat_rad)*np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))
dist_km = R * c 
sign = np.where(df_data_list['lat'] < centroid_lat, -1, 1)   # south → –1, north → +1
df_data_list['dist_km'] = dist_km * sign
df_data_list.rename(columns={0: 'wrf_day', 1: 'wrf-ctsm_day', 2: 'modis_day', 3: 'viirs_day', 
                             4:'wrf_night', 5: 'wrf-ctsm_night', 6: 'modis_night', 7: 'viirs_night'}, inplace=True)
df_data_list.to_csv(f'./data_for_figure/section_data_{date}.csv', index=False)
df_data_list.head()             

,wrf_day,wrf-ctsm_day,modis_day,viirs_day,wrf_night,wrf-ctsm_night,modis_night,viirs_night,lat,lon,index_lat,index_lon,dist_km
0,37.000473,34.100204,29.470000000000027,32.93,7.587372,10.037094,9.690000000000055,7.91,53.274712,-2.582367,0,11,-32.180946
1,36.858887,34.2556,31.03000000000003,32.67,7.67572,9.963104,nan,NaN,53.285488,-2.564331,1,12,-30.485749
2,37.279449,34.630859,30.33000000000004,31.55,7.392639,10.351852,nan,NaN,53.296276,-2.546295,2,13,-28.789862
3,36.511566,34.1427,30.450000000000045,32.05,7.706955,9.387085,nan,NaN,53.307060,-2.528259,3,14,-27.094489
4,37.050995,33.263794,30.130000000000052,30.99,6.242111,8.455856,9.050000000000011,NaN,53.317844,-2.510223,4,15,-25.399330


In [22]:
i1, i2 = 23, 36 # lon
j1, j2 = 12, 25 # lat
southwest_point = df_data_list[(df_data_list['index_lat']==j1) & (df_data_list['index_lon']==i1)].copy()
southwest_point

,wrf_day,wrf-ctsm_day,modis_day,viirs_day,wrf_night,wrf-ctsm_night,modis_night,viirs_night,lat,lon,index_lat,index_lon,dist_km
12,45.980423,45.108459,41.31,42.85,25.988937,20.717896,nan,21.85,53.404129,-2.365997,12,23,-11.847675


In [25]:
northeast_point = df_data_list[(df_data_list['index_lat']==j2) & (df_data_list['index_lon']==i2)].copy()
northeast_point

,wrf_day,wrf-ctsm_day,modis_day,viirs_day,wrf_night,wrf-ctsm_night,modis_night,viirs_night,lat,lon,index_lat,index_lon,dist_km
25,32.327118,33.147507,35.25000000000006,35.05,7.603638,9.281067,9.57000000000005,10.13,53.544334,-2.131653,25,36,10.142986
